# Design Doc - Telepresence Robot

## Goals

Create a robot which can be driven from anywhere around the world over a video feed. Try to achieve minimum possible latency to maximize user experience.

## Components

* Robot (physical)
* Robot controller electronics
* Power source (battery)
* Robot firmware
* Controller frontend
* Controller backend
* Low latency backend (edge backend?)

## Robot Chassis

Design goals: keep it cheap and simple.

* Ideally tank drive -esque drivetrain
* Differential torque control
* Minimum 3d printing, use Al extrusion instead

### Drivetrain

Two powertrains for each side. Each powertrain contains one motor and one driven wheel and is mostly self-contained.

#### Powertrain

* **Manufacturing**: 3d printed
* **Quantity**: 2 units per robot
* **Motor Type**: NEMA17 stepper motor (1.7inch)
    * [Jameco](https://www.jameco.com/z/STP-42D201-37-Shinano-Kenshi-12-Volt-1-8-Step-Angle-Bipolar-Stepper-Motor_2158531.html)
    * Stock: 10 units in Yash's lab
    * Supply chain: impossible to source, find replacement
    * Holding torque: 3200g-cm / 32g-m
    * Torque curve TBD
* **Gearing**: tune max robot speed to knee speed of the motor (peak power)
    * Calculate peak power, $P\propto I\times\omega$
    * Gear ratio = $\frac{\omega_\text{knee}}{\omega_\text{output-max}}$
* Other notes: may need to switch to better stepper motors if these are not performant enough. May get expensive.

#### Casters

Two caster wheels will be used in the front corners of the robot to allow for steering input

## Electronics

The electronics consist of a motherboard and several system on modules.

* **Motherboard**: connects everything together. Hosts power management, voltage regulation, signal routing and signal isolation
* **INA228**: battery PMIC to ensure safety handling lithium polymer, lithium ion and other chemistry batteries
* **Power FETs**: enable/disable high voltage power electronics, enable/disable compute power
* **ISO7762DBQ**: logic-level isolator. Allows different voltage buses to power compute and motor drivers for safety
* **TMC5160T Pro**: SOM, BigTreeTech SilentStepStick bipolar stepper driver board, fully integrated
* **ESP32 C5**: Main CPU, has 2.4GHz and 5GHz WiFi support, with AX support and OFDMA
* **Isolators**: multiple open-collector isolators are used to control power electronics and slow signals (Vishay VO617A-2)

### Electronics architecture

The main controller which connects everything together is the ESP32 C5. It uses various buses to commnunicate with peripherals.

An i2c bus is used to communicate with the INA228. The INA228 is a differential sense amplifier and high voltage analog to digital converter. It has a few jobs:

1. Measure the instantaneous current being consumed from the battery
2. Measure the voltage of the battery
3. Coulomb count (integrate the amps drawn over time to measure battery usage in ampere-hour capacity)

An SPI bus is used to communicate with the TMC5160 motor controllers. The TMC5160 motor controllers are chained together, SDO to SDI.

There are three GPIOs used to enable/disable different power subsystems:

1. GPIO24: setting this as high will enable the MOSFET which powers the high voltage section
2. GPIO23: setting this as high will enable the output MOSFETs of the motor drivers (this brings up the enable pin of the TMC5160 drivers)
3. GPIO4/LPGPIO4: setting this as high will enable power to the compute subsystem. Disabling this will fully shut down the robot

## Power

This board is designed to be powered by a rechargeable battery. This board is designed such that longer inductive spikes should be able to recharge the battery and TVS operation should be minimized. Achieving max power on a switching power supply will result in possible magic smoke in the TMC5160 motor drivers as well as its FETs.

Note: the SilentStepStick boards have a design flaw which results in the max operating voltage of 40V instead of the 50+V which are often advertised.

Two MOSFETs which power up the two different power domains:

1. High voltage power domain: provides power to motors
2. Low voltage (aux) power domain: provides power to the compute section (microcontroller, isolators, Vio, etc.)

The aux power output is designed to power a buck converter to bring down the working voltage closer to ~6-8v to not blow up the LDOs. Under normal operation, power up the board using that LDO via the IOLV header. Bring GPIO4/LPGPIO4 to high to ensure the rail does not die.

To allow for soft power up, a header exists to control this power supply: **J2 / EXT_PWR_BTN**. While `EXT_PWR_BTN` is shorted, it will force the low voltage FET into conduction allowing the robot to boot up.

Rapidly set GPIO4/LPGPIO4 to high on ESP32 boot to ensure power stays on.

## Motor Control

This board is designed around the TCM5160 chips to drive stepper motors. The TMC5160 contains a "ramp generator" - an ASIC which can internally generate a signal to "step" the motor (commutate it so it moves). The TMC5160 can offload most of the work off the microcontroller to its own ASICs allowing for easy development.

The TMC5160 motor drivers are connected via SPI and chained together. An infinite number (minus real-life timing issues) of drivers can be chained together.

Read the TMC5160 stepper driver's datasheet for implementation details.

## Communications

The primary robot controller is an ESP32-C5, a microcontroller with dual band (2.4GHz and 5GHz) wifi 6 (AX) with OFDMA support for lower latency.

To facilitate client to robot controller, multiple hardware and software components will be utilized:

* **Client**: web browser (Next.js export / React), deployed to CloudFlare
* **Backend**: TypeScript + WebSocket + REST, deployed to cloud
* **Low-latency (edge) backend (LLBE)**: WebRTC + DTLS, deployed to edge server (Raspberry Pi 5, ethernet)
* **LLBE to robot communication**: DTLS (on ESP32-C5)

Additionally, video streaming will rely WebRTC.

WebRTC connections will be established via STUN.